# Importing Libraries

We load a randoms catalogue and transform from ICRS to cartesian coordinates. After, we use it to estimate the survey window function.

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt

# ICRS to NGC and SGC in cartesian coordinates

We first define two function that are useful. The first one take an usual survey data as input and split it into NGC and SGC. The second is used to transform sky coordinates into cartesian coordinates, which are used for power spectrum estimations:

In [ ]:
def GalacticCapSplit(data, cosmo):
    # Convert to Galactic coordinates using astropy
    coords = SkyCoord(ra=data['RA']*u.degree, dec=data['DEC']*u.degree, frame='icrs')
    galactic_coords = coords.galactic
    
    # Split the data into NGC and SGC using Galactic latitude (b)
    ngc_mask = galactic_coords.b.deg > 0  # NGC: b > 0
    sgc_mask = galactic_coords.b.deg < 0  # SGC: b < 0

    return data[ngc_mask], data[sgc_mask]

def sky_to_cartesian(data, cosmo):
    ra = data['RA']
    dec = data['DEC']
    z = data['Z']
    
    # Convert to Cartesian coordinates
    ra_rad = np.deg2rad(ra)
    dec_rad = np.deg2rad(dec)
    comoving_distance = cosmo.comoving_distance(z).value  # in Mpc
    
    x = comoving_distance * np.cos(dec_rad) * np.cos(ra_rad)
    y = comoving_distance * np.cos(dec_rad) * np.sin(ra_rad)
    z = comoving_distance * np.sin(dec_rad)
    
    return [x,y,z]

def ReadFits(fn):
    with fits.open(fn) as hdul:
        # Assuming the data is in the first extension (index 1)
        data = hdul[1].data
    return data

In [ ]:
# Path to your FITS file
file_path = 'data/DESI/QSO/catalogues/'
fn_random = 'QSO_ffa_NGC_0_clustering.ran.fits'
fn_data = 'QSO_ffa_NGC_clustering.dat.fits'

# Specify the cosmology. Using the DESI cosmology
cosmo = FlatLambdaCDM(H0=67.36, Om0=0.3137721026737606)

In [ ]:
#read the files
randoms = ReadFits(file_path+fn_random)
data = ReadFits(file_path+fn_data)


#Split each into NGC and SGC
randoms_NGC,randoms_SGC = GalacticCapSplit(randoms, cosmo)
data_NGC,data_SGC = GalacticCapSplit(data, cosmo)

#Delete the original
del(randoms,data)

The catalogues we loaded in this example were already splitted into NGC and SGC, and we are using the former so the resulting SGC should be empty:

In [ ]:
data_SGC

In [ ]:
data_NGC_positions = sky_to_cartesian(data_NGC,cosmo)

In [ ]:
randoms_NGC_positions = sky_to_cartesian(randoms_NGC,cosmo)

In [ ]:
randoms_NGC_weights = randoms_NGC['Weight']

In [ ]:
data_NGC_weights = data_NGC['Weight']

# Estimating data power spectrum

In [ ]:
results = CatalogFFTPower(data_positions1=data_NGC_positions, data_weights1=data_NGC_weights, randoms_positions1=randoms_NGC_positions, randoms_weights1=randoms_NGC_weights,
                        nmesh=Nmesh, resampler='tsc', boxsize = L, interlacing=2, ells=(0,2,4), los='firstpoint', edges=kedges, position_type='xyz', dtype='f8',mpiroot=0)

In [ ]:
result.poles.save('test.npy')

In [ ]:
k1, p0_1 = results_1.poles(ell=0, complex = False, return_k = True)
k2, p0_2 = results_2.poles(ell=0, complex = False, return_k = True)

In [ ]:
plt.plot(k2, k2*p0_2)
plt.plot(k1, k1*p0_1)